In [2]:

# Import required libraries
import re         # For regular expressions (text tokenization)
import requests   # For downloading the corpus
import gzip       # For decompressing the downloaded corpus
import io         # For handling byte streams
import math       # For mathematical operations (log, exp)
import random     # For random number generation
from collections import defaultdict  # For efficient dictionary operations
import pickle, os # For saving and loading the model

In [3]:
import re, torch, torch.nn as nn

torch.manual_seed(42)

docs = [
    'Movies are fun for everyone.',
    'Watching movies is great fun.',
    'Enjoyed a great movie today.',
    'Research is interesting and important.',
    'Learning math is very important.',
    'Science discovery is interesting.',
    'Rock is great to listen to.',
    'Listen to music for fun.',
    'Music is fun for everyone.',
    'Listen to folk music.'
]

labels = [1, 1, 1, 3, 3, 3, 2, 2, 2, 2]
num_classes = len(set(labels))

In [4]:
def tokenize(text):
    return re.findall(r"\w+" , text.lower())

def get_vocabulary(texts):
    tokens = {token for text in texts for token in tokenize(text)}
    return {word: idx for idx, word in enumerate(sorted(tokens))}

vocabulary = get_vocabulary(docs)


In [5]:
def doc_to_bow(doc, vocabulary):
    tokens = set(tokenize(doc))
    bow = [0] * len(vocabulary)
    for token in tokens:
        if token in vocabulary:
            bow[vocabulary[token]] = 1
    return bow

In [6]:
vectors = torch.tensor(
    [doc_to_bow(doc, vocabulary) for doc in docs],
    dtype=torch.float32
)

labels = torch.tensor(labels, dtype=torch.long) - 1

In [7]:
input_dim = len(vocabulary)
hidden_dim = 50
output_dim = num_classes

class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = SimpleClassifier(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

for step in range(3000):
    optimizer.zero_grad()
    loss = criterion(model(vectors), labels)
    loss.backward()
    optimizer.step()

In [9]:
new_docs =[
"Listening to rock music is fun.",
"I love science very much."]

class_names = ["Cinema", "Music","Science"]
new_doc_vectors = torch.tensor([doc_to_bow(new_doc, vocabulary) for new_doc in new_docs], dtype=torch.float32)

with torch.no_grad():
    outputs = model(new_doc_vectors)
    predicted_ids = torch.argmax(outputs, dim=1) + 1

for i, new_doc in enumerate(new_docs):
    print(f'{new_doc}: {class_names[predicted_ids[i].item() - 1]}')

Listening to rock music is fun.: Music
I love science very much.: Science
